# Overview
This notebook reads the sub-dataset from WikiTables json files and parse them to a DataFrame.
Then save the dataframe to a csv file for the other nb to start with it.

## Reading Data Json Files

In [ ]:
import json
import os
import re

def read_index_file(index_file_path):
    try:
        with open(index_file_path, 'r') as file:
            index_data = json.load(file)
        return index_data
    except FileNotFoundError:
        print(f"Index file {index_file_path} not found.")
        return None

def read_json_files_from_index(index_data, folder_path):
    json_files_data = {}
    
    for key, value in index_data.items():
        json_file_name = f"{value}.json"
        json_file_path = os.path.join(folder_path, json_file_name)
        
        try:
            with open(json_file_path, 'r') as json_file:
                json_data = json.load(json_file)
            json_files_data[key] = json_data
        except FileNotFoundError:
            print(f"File {json_file_name} not found in {folder_path}.")
    
    return json_files_data

In [2]:
index_file_path = 'data/orig_id_to_uid.json'
index_data = read_index_file(index_file_path)
index_data

{'7259': 'List_of_awards_won_by_Andrei_Tarkovsky_0',
 '9613': 'Al_Sadd_SC_2',
 '4401': 'List_of_politicians,_lawyers,_and_civil_servants_educated_at_Jesus_College,_Oxford_2',
 '5807': "Looney_Tunes_Super_Stars'_Bugs_Bunny:_Hare_Extraordinaire_0",
 '12593': 'List_of_NCAA_conferences_2',
 '4231': 'List_of_Places_of_Scenic_Beauty_of_Japan_(Fukuoka)_2',
 '2350': 'Indian_general_election_in_Madras,_1957_0',
 '9662': '2009_K_League_0',
 '519': 'List_of_video_games_published_by_Aksys_Games_9',
 '14018': 'Media_in_the_Fraser_Valley_0',
 '15058': 'Aamir_Ali_0',
 '6352': 'List_of_winners_of_the_Boston_Marathon_0',
 '8223': 'List_of_University_of_Texas_at_Austin_alumni_5',
 '8242': 'List_of_battles_by_casualties_0',
 '4493': 'List_of_South_African_airports_by_passenger_movements_5',
 '5602': 'List_of_New_York_University_faculty_and_staff_5',
 '1712': 'List_of_named_passenger_trains_of_the_United_States_(S-Z)_0',
 '10458': 'List_of_populated_places_in_Adilabad_district_3',
 '1943': 'Live_with_Regi

In [4]:
json_files_folder = 'data/tables_tok'
json_files_data = read_json_files_from_index(index_data, json_files_folder)

In [5]:
first_key = list(json_files_data.keys())[0]
first_value = json_files_data[first_key]
first_value.keys()

dict_keys(['url', 'title', 'header', 'data', 'section_title', 'section_text', 'uid', 'intro'])

## Cleaning Attribute Titles

In [10]:
from inflect import engine

inflect_engine = engine()

def clean_attribute_name(attribute_name):
    extra_words = ['list', 'table']

    # Step 1: Remove extra spaces (leading, trailing, and multiple spaces between words)
    cleaned_name = re.sub(r'\s+', ' ', attribute_name.strip())
    
    # Step 2: Replace underscores with spaces to handle compound words
    cleaned_name = cleaned_name.replace('_', ' ')
    
    # Step 3: Singularize each word
    cleaned_name = ' '.join([inflect_engine.singular_noun(word) or word for word in cleaned_name.split()])
    
    # Step 5: Remove Extra Parenthesis
    cleaned_name = re.sub(r"\s*\(.*?\)", "", cleaned_name).strip()
    
    # Step 4: Remove extra words
    for word in extra_words:
        cleaned_name = re.sub(rf'\b{word}\b', '', cleaned_name, flags=re.IGNORECASE).strip()
    
    if not cleaned_name or cleaned_name.strip() == '':
        cleaned_name = '#'

    return cleaned_name

## Building the DataFrame

In [11]:
import pandas as pd
import json

data_for_df = []

for key, value in json_files_data.items():
    schema = [clean_attribute_name(item[0]) for item in value['header']]
    data_json = json.dumps(value['data'])

    data_for_df.append({
        "id": key,
        "uid": value['uid'],
        "url": value['url'],
        "title": value['title'],
        "section_title": value['section_title'],
        "section_text": value['section_text'],
        "intro": value['intro'],
        "schema": schema,
        "data": data_json
    })

df = pd.DataFrame(data_for_df)
df

,id,uid,url,title,section_title,section_text,intro,schema,data
0,7259,List_of_awards_won_by_Andrei_Tarkovsky_0,https://en.wikipedia.org/wiki/List_of_awards_w...,List of awards won by Andrei Tarkovsky,Film awards and nominations -- Cannes Film Fes...,,Andrei Tarkovsky (1932-1986) was a Soviet film...,"[Year, Award, Result, Category/Recipient]","[[[""1969"", [""/wiki/1969_in_film""]], [""FIPRESCI..."
1,9613,Al_Sadd_SC_2,https://en.wikipedia.org/wiki/Al_Sadd_SC,Al Sadd SC,,,Al-Sadd Sports Club (Arabic: نادي السد الرياضي...,"[Player, Nationality, Int . cap / goal, Club c...","[[[""Mubarak Anber"", [""/wiki/Mubarak_Anber""]], ..."
2,4401,"List_of_politicians,_lawyers,_and_civil_servan...",https://en.wikipedia.org/wiki/List_of_politici...,"List of politicians, lawyers, and civil servan...",Alumni -- Judges,Viscount Sankey,Jesus College is one of the constituent colleg...,"[Name, M, G, Degree, Note]","[[[""Austin Amissah"", [""/wiki/Austin_Amissah""]]..."
3,5807,Looney_Tunes_Super_Stars'_Bugs_Bunny:_Hare_Ext...,https://en.wikipedia.org/wiki/Looney_Tunes_Sup...,Looney Tunes Super Stars' Bugs Bunny: Hare Ext...,Contents,All cartoons on this disc star Bugs Bunny .,Looney Tunes Super Stars' Bugs Bunny: Hare Ext...,"[#, Title, Co-Star, Release Date, Director, Se...","[[[""1"", []], [""Mutiny on the Bunny"", [""/wiki/M..."
4,12593,List_of_NCAA_conferences_2,https://en.wikipedia.org/wiki/List_of_NCAA_con...,List of NCAA conferences,"NCAA Division I -- Non-football , multi-sport ...",,The National Collegiate Athletic Association (...,"[Conference, Nickname, Founded, Member, Sport,...","[[[""America East Conference"", [""/wiki/America_..."
...,...,...,...,...,...,...,...,...,...
15268,9280,List_of_Virtual_Console_games_for_Wii_U_(North...,https://en.wikipedia.org/wiki/List_of_Virtual_...,List of Virtual Console games for Wii U (North...,Available titles -- Game Boy Advance,These titles were originally released for use ...,This is a list of Virtual Console games that a...,"[Title, Publisher, Developer, Release Date, ESRB]","[[[""Advance Wars"", [""/wiki/Advance_Wars""]], [""..."
15269,7000,List_of_football_clubs_in_the_German_national_...,https://en.wikipedia.org/wiki/List_of_clubs_in...,List of clubs in the German football championship,List -- P,,This is a list of all clubs that have taken pa...,"[Club, Qualified, Champion, Runners-up, Season]","[[[""DFC Prag"", [""/wiki/DFC_Prag""]], [""1"", []],..."
15270,936,National_Register_of_Historic_Places_listings_...,https://en.wikipedia.org/wiki/National_Registe...,National Register of Historic Places listings ...,Current listings,,This is a list of the National Register of His...,"[#, Name on the Register, Date listed, Locatio...","[[[""1"", []], [""Ames Creek Bridge"", [""/wiki/Ame..."
15271,6479,Iowa_Energy_0,https://en.wikipedia.org/wiki/Iowa_Wolves,Iowa Wolves,Players -- NBA call-ups,Anthony Tolliver was called up to the New Orle...,The Iowa Wolves are an American professional b...,"[Season, Player, NBA team, Date called up and ...","[[[""2008-09"", [""/wiki/2008\u201309_NBA_Develop..."


In [12]:
df.to_csv('wikitables.csv', index=False)